In [1]:
#import packages
import pandas as pd
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#modify import path
import sys
sys.path.append('../our_analysis')

#import our code
from hybrid_mle import fit_hybrid_mixed_model_decayingMF

In [3]:
#Set WD
notebook_dir = os.getcwd()

In [ ]:
#load dataset
df = pd.read_csv("../dataset/all_subjects.csv") #our dataset
# df = pd.read_csv("../dataset/beh_noslow.csv") #i'm fitting to replicate the paper.

#Let's remove all of the slow trials for the df
df = df[df['slow']==0]
# We need to cast the choice data to an int
df['choice1'] = df['choice1'].astype(int)
df['choice2'] = df['choice2'].astype(int)
df['final_state'] = df['final_state'].astype(int)

story_trials = df[df["condition"] == "story"]
abstract_trials=df[df["condition"] == "abstract"]

In [ ]:
story_results_df, params, logli_story_basic = fit_hybrid_mixed_model_decayingMF(
    data_df=story_trials,
    stan_file="../our_analysis/decaying_mf_weight.stan",
    output_file="story_params_decay.csv"
)

abstract_results_df, params, logli_abstract_basic = fit_hybrid_mixed_model_decayingMF(
    data_df=abstract_trials,
    stan_file="../our_analysis/decaying_mf_weight.stan",
    output_file="abstract_params_decay.csv"
)

In [ ]:
#store the results in a clean table we can use for the assignment
results =[] 

for df in [story_results_df,abstract_results_df]:
    params=df.drop(columns='w').iloc[0].to_dict()
    params['w_mean'] = df['w'].mean()
    params['w_std'] = df['w'].std()
    results.append(params)

results[0]['logli']=logli_story_basic
results[1]['logli']=logli_abstract_basic

results_df = pd.DataFrame(results)
results_df.head()


results_df.to_csv("hybrid_model_params_decayingMF.csv", index=False)


In [8]:
results_df.head()

,participant,condition,alpha1,alpha2,lmbd,beta1,beta2,p,decay_rate,w_mean,w_std,logli
0,1,story,0.028954,0.867799,0.999670,8.61319,1.98531,0.016556,0.671942,0.434942,0.351164,-7439.72
1,5,abstract,0.353279,0.492359,0.646353,5.68530,3.01351,0.042321,0.987428,0.532249,0.272295,-7006.55


# BIC & AIC Calc

In [ ]:
import pandas as pd
formatting_df = pd.read_csv("/Users/nokada/Documents/CALTECH/sdn_hw2/part4/hybrid_model_params_decayingMF.csv")

num_params = 8
story_sample_size = 46
abstract_sample_size = 48

AIC_story = 2*num_params-2*formatting_df['logli'][0]
AIC_abstract = 2*num_params-2*formatting_df['logli'][1]

BIC_story = 2*num_params * np.log(story_sample_size) - 2 * formatting_df['logli'][0]
BIC_abstract = 2*num_params * np.log(abstract_sample_size) - 2 * formatting_df['logli'][1]



formatting_df['AIC'] = AIC_story
formatting_df['AIC'][1] = AIC_abstract

formatting_df['BIC'] = BIC_story
formatting_df['BIC'][1] = BIC_abstract

In [12]:
formatting_df = formatting_df.round(3)
formatting_df.to_csv("../csvs_formatted/hybrid_model_params_decayingMF.csv", index=False)


In [13]:
formatting_df.head()

,participant,condition,alpha1,alpha2,lmbd,beta1,beta2,p,decay_rate,w_mean,w_std,logli,AIC,BIC
0,1,story,0.029,0.868,1.000,8.613,1.985,0.017,0.672,0.435,0.351,-7439.72,14895.44,14940.698
1,5,abstract,0.353,0.492,0.646,5.685,3.014,0.042,0.987,0.532,0.272,-7006.55,14029.10,14075.039
